In [1]:
import zipfile
from tqdm import tqdm

# Open the ZIP file
with zipfile.ZipFile('/root/yolov8_dataset_5297.zip', 'r') as zip_ref:
    # Get the list of files in the ZIP archive
    file_list = zip_ref.namelist()
    
    # Initialize tqdm progress bar
    with tqdm(total=len(file_list), desc="Extracting files") as pbar:
        # Extract each file
        for file in file_list:
            zip_ref.extract(file, '/root/H')
            # Update the progress bar
            pbar.update(1)

Extracting files: 100%|██████████| 10604/10604 [00:11<00:00, 891.17it/s]


In [3]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import os
import random
import pandas as pd
import matplotlib.image as mpimg
import seaborn as sns
import torch
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
height = 640
width = 640

def train_model():
    # Check CUDA availability inside the function
    cuda_available = torch.cuda.is_available()
    device = 'cuda' if cuda_available else 'cpu'
    print(f"Using device: {device}")

    model = YOLO('yolov8n.pt')
    results = model.train(
        data="/root/yolov8_dataset_5297/data.yaml",
        epochs=900,
        imgsz=(height, width),
        seed=42,
        batch=16 if cuda_available else 4,  # Adjust batch size based on device
        workers=8 if cuda_available else 2,  # Adjust workers based on device
        device=0 if cuda_available else 'cpu',  # Uses all available GPUs if CUDA is available
        patience=100,
        lr0=0.01,
        lrf=0.001,
        amp=True  # Enable mixed precision training
    )
    return results

if __name__ == '__main__':
    results = train_model()
    print("Training complete. Results:", results)

In [ ]:
import matplotlib.pyplot as plt
import os
import cv2
model = YOLO("/root/runs/detect/train/weights/last.pt")
def detect_records(input_folder):
    # Iterate over all images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path)
            results = model(img)
            for result in results:
                if result.boxes.data.shape[0] > 0:  # Check for detections+
                    for i, box in enumerate(result.boxes.data.tolist()):
                        xmin, ymin, xmax, ymax, conf, cls = box

                        # Draw the bounding box on the image
                        cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 5)

            plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            plt.title(f"Detections on {filename}")
            plt.axis('off')
            plt.show()

input_folder = "/root/yolov8_dataset_5297/test/images"
detect_records(input_folder)